In [ ]:
# Librerias
import pandas as pd
import numpy as np
import hashlib 
import logging
from pathlib import Path
import sys
from io import StringIO
import unicodedata 
import re 

In [ ]:
"""
Script para:
1. Convertir nombres de las columnas del archivo  salubridad_restaurantes a minúsculas .
2. Crear dimensión de tipos de cocina con ID hash alfanumérico de 4 dígitos.

"""

# --- Configuración de Logging  ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s.%(funcName)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
log = logging.getLogger(__name__)

# Definimos las rutas con las que bamos a trabajar 
try:
    RUTA_BASE = Path(r"H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS").resolve()
    RUTA_PROCESSED = RUTA_BASE / "data" / "processed"
    RUTA_DIM = RUTA_PROCESSED / "dim"
    RUTA_SALUBRIDAD_PROC = (RUTA_PROCESSED / "salubridad_restaurantes_con_ids.csv").resolve()
    RUTA_SALIDA_DIM_COCINA = (RUTA_DIM / "dim_descripcion_cocina.csv").resolve()
    RUTA_PROCESSED.mkdir(parents=True, exist_ok=True)
    RUTA_DIM.mkdir(parents=True, exist_ok=True)
except Exception as e:
    log.exception(f"Error crítico al definir rutas: {e}")
    sys.exit(1)

COLUMNA_NOMBRE_RESTAURANTE = "nombre_restaurante"
COLUMNA_COCINA = "tipo_cocina" # <-- Usando el nombre corregido

# --- Funciones Auxiliares ---

# Por sia caso normalisamos los nombres de las columnas
def limpiar_texto_para_hash(texto: str) -> str | None:
    """Limpia y estandariza texto antes de generar un hash."""
    if pd.isna(texto) or not isinstance(texto, str): return None
    try:
        
        norm = texto.lower()
        norm = unicodedata.normalize('NFKD', norm).encode('ASCII', 'ignore').decode('utf-8')
        norm = re.sub(r'[^\w\s]+', '', norm) # Quitar puntuación etc.
        norm = re.sub(r'\s+', ' ', norm).strip() # Normalizar espacios
        return norm if norm else None
    except Exception as e:
        log.warning(f"Error limpiando texto '{texto}': {e}")
        return None

"""Creamos una funcion para crear las ids de archivo dim
scripcion_cocina, que son un hash corto de 4 digitos hexadecimales.
El hash se genera a partir de la descripción de cocina, y se le añade un prefijo "COC" para identificarlo como un ID de cocina.
"""
def generar_id_cocina(descripcion_cocina: str) -> str | None:
 
    if pd.isna(descripcion_cocina) or not str(descripcion_cocina).strip():
        return None

    # Limpiar/Estandarizar la descripción antes de hashear para consistencia
    texto_limpio = limpiar_texto_para_hash(str(descripcion_cocina))
    if not texto_limpio:
        log.debug(f"Descripción cocina '{descripcion_cocina}' resultó vacía tras limpiar.")
        return None # No generar ID si el nombre limpio es vacío

    try:
        # Generar hash MD5, tomar primeros 4 caracteres hexadecimales
        hash_id = hashlib.md5(texto_limpio.encode('utf-8')).hexdigest()[:4].upper()
        return f"COC{hash_id}" # Añadir prefijo "COC"
    except Exception as e:
        log.warning(f"No se pudo generar ID para cocina '{descripcion_cocina}': {e}.")
        return None

# --- Funciones Principales ---
"""CargaMOS un archivo  CSV procesado."""
def cargar_datos_procesados(ruta_archivo: Path) -> pd.DataFrame | None:
    
    # ... (igual que antes) ...
    log.info(f"Cargando archivo procesado desde: {ruta_archivo}")
    if not ruta_archivo.is_file(): log.error(f"Archivo no encontrado: {ruta_archivo}"); return None
    try:
        df = pd.read_csv(ruta_archivo, low_memory=False)
        log.info(f"Archivo cargado: {df.shape[0]} filas, {df.shape[1]} columnas.")
        return df
    except Exception as e: log.exception(f"Error al cargar CSV desde '{ruta_archivo}': {e}"); return None

"""Convertimos nombres a minúsculas y guarda el DataFrame."""
def nombres_a_minusculas_y_guardar(df: pd.DataFrame, columna_nombre: str, ruta_salida: Path) -> bool:
    log.info(f"Convirtiendo '{columna_nombre}' a minúsculas...")
    if columna_nombre not in df.columns: log.error(f"Columna '{columna_nombre}' no existe."); return False
    try:
        df_modificado = df.copy()
        df_modificado[columna_nombre] = df_modificado[columna_nombre].astype(str).str.lower()
        log.info("Conversión a minúsculas OK.")
        log.info(f"Guardando DataFrame modificado en: {ruta_salida}")
        df_modificado.to_csv(ruta_salida, index=False, encoding='utf-8-sig')
        log.info("Archivo guardado con nombres en minúsculas.")
        return True
    except Exception as e: log.exception(f"Error en minúsculas/guardado en '{ruta_salida}': {e}"); return False

# *** FUNCIÓN MODIFICADA para usar IDs HASH ***

"""
Extraemos los tipos de cocina únicos, creamos una dimensión con IDs HASH de 4 dígitos
(prefijo COC) y la guarda, manejando colisiones.    
"""
def crear_y_guardar_dimension_cocina(df: pd.DataFrame, columna_cocina: str, ruta_salida_dim: Path) -> bool:
   
    log.info(f"Creando dimensión desde columna '{columna_cocina}' con IDs hash...")
    if columna_cocina not in df.columns:
        log.error(f"Columna '{columna_cocina}' no encontrada."); return False

    try:
        # 1. Extraer, limpiar, obtener únicos y ordenar descripciones
        descripciones_unicas = df[columna_cocina].dropna().astype(str).unique()
        descripciones_unicas_ordenadas = sorted([desc.strip() for desc in descripciones_unicas if str(desc).strip()])

        if not descripciones_unicas_ordenadas:
            log.warning(f"No se encontraron descripciones válidas en '{columna_cocina}'."); return False

        log.info(f"Encontrados {len(descripciones_unicas_ordenadas)} tipos de cocina únicos.")

        # 2. Generar IDs para cada descripción única
        registros_dim = []
        for desc in descripciones_unicas_ordenadas:
            id_cocina_gen = generar_id_cocina(desc)
            if id_cocina_gen: # Solo añadir si se pudo generar ID
                registros_dim.append({'id_cocina': id_cocina_gen, 'descripcion_cocina': desc})
            else:
                log.warning(f"No se generó ID para la descripción: '{desc}'")

        if not registros_dim:
             log.error("No se pudieron generar IDs válidos para ninguna descripción de cocina.")
             return False

        # 3. Creamos DataFrame inicial de la dimensión
        dim_cocina = pd.DataFrame(registros_dim)
        log.info(f"Generados {len(dim_cocina)} registros iniciales para la dimensión.")

        # 4. Verificamos como manejar colisiones de ID 
        id_counts = dim_cocina['id_cocina'].value_counts()
        colisiones = id_counts[id_counts > 1]

        if not colisiones.empty:
            log.warning(f"¡COLISIÓN DE HASH DETECTADA! {len(colisiones)} IDs generados están repetidos:")
            # Mostrar qué IDs colisionaron y qué descripciones los generaron
            ids_colisionados = colisiones.index.tolist()
            desc_colisionadas = dim_cocina[dim_cocina['id_cocina'].isin(ids_colisionados)]
            log.warning(f"Detalle de colisiones:\n{desc_colisionadas.sort_values(by='id_cocina').to_string()}")
            # Eliminamos duplicados, manteniendo el primero
            rows_antes = len(dim_cocina)
            dim_cocina.drop_duplicates(subset=['id_cocina'], keep='first', inplace=True)
            rows_despues = len(dim_cocina)
            log.warning(f"Se eliminaron {rows_antes - rows_despues} filas debido a colisiones de ID.")
        else:
            log.info("No se detectaron colisiones en los IDs de cocina generados.")

        # 5. Convertimos tipos de datos
        dim_cocina['id_cocina'] = dim_cocina['id_cocina'].astype('string') # Guardar hash como string
        dim_cocina['descripcion_cocina'] = dim_cocina['descripcion_cocina'].astype('string')

        # 6. Guardamos el resultado
        log.info(f"Guardando dimensión de cocina ({len(dim_cocina)} filas) en: {ruta_salida_dim}")
        ruta_salida_dim.parent.mkdir(parents=True, exist_ok=True)
        dim_cocina.sort_values(by='descripcion_cocina', inplace=True) # Ordenar alfabéticamente por descripción
        dim_cocina.to_csv(ruta_salida_dim, index=False, encoding='utf-8-sig')
        log.info("Dimensión de cocina guardada exitosamente.")
        return True

    except Exception as e:
        log.exception(f"Error creando o guardando la dimensión de cocina: {e}")
        return False

# --- Flujo Principal ---
def main():
    """Orquesta la carga, modificación y creación de dimensión."""
    log.info("================================================")
    log.info("=== INICIO: Modificar Nombres y Crear Dim Cocina (ID Hash) ===")
    log.info("================================================")

    # 1. Validar archivo de entrada
    log.info(f"Verificando archivo de entrada: {RUTA_SALUBRIDAD_PROC}")
    if not RUTA_SALUBRIDAD_PROC.is_file():
        log.critical(f"Abortando: Archivo no encontrado: {RUTA_SALUBRIDAD_PROC}"); sys.exit(1)
    log.info("Archivo de entrada encontrado.")

    # 2. Cargar datos
    df_salubridad = cargar_datos_procesados(RUTA_SALUBRIDAD_PROC)
    if df_salubridad is None or df_salubridad.empty:
        log.critical("Fallo al cargar datos procesados. Abortando."); sys.exit(1)

    df_para_dimension = df_salubridad.copy() # Usar copia para crear dimensión
    df_para_modificar = df_salubridad      # Referencia para modificar y guardar

    # 3. Tarea 1: Convertir nombres a minúsculas y guardar 
    log.info("--- Tarea 1: Convertir Nombres a Minúsculas y Guardar ---")
    guardado_ok = nombres_a_minusculas_y_guardar(
        df_para_modificar, COLUMNA_NOMBRE_RESTAURANTE, RUTA_SALUBRIDAD_PROC
    )
    if not guardado_ok:
        log.error("Fallo Tarea 1 (minúsculas/guardar). Continuando con dimensión desde datos originales.")

    # 4. Tarea 2: Crear dimensión de cocina (usando la copia original cargada)
    log.info("--- Tarea 2: Crear Dimensión de Cocina (ID Hash) ---")
    creacion_dim_ok = crear_y_guardar_dimension_cocina(
        df_para_dimension,   # Usamos la copia cargada originalmente
        COLUMNA_COCINA,
        RUTA_SALIDA_DIM_COCINA
    )

    log.info("================================================")
    if guardado_ok and creacion_dim_ok: log.info("=== Proceso completado exitosamente ===")
    else: log.warning("=== Proceso completado con errores (revisar logs) ===")
    log.info(f"Archivo modificado: {RUTA_SALUBRIDAD_PROC}")
    log.info(f"Dimensión cocina creada/actualizada en: {RUTA_SALIDA_DIM_COCINA}")
    log.info("================================================")

if __name__ == "__main__":
    main()

2025-04-19 11:33:38 [INFO] __main__.main: ================================================
2025-04-19 11:33:38 [INFO] __main__.main: === INICIO: Modificar Nombres y Crear Dim Cocina ===
2025-04-19 11:33:38 [INFO] __main__.main: ================================================
2025-04-19 11:33:38 [INFO] __main__.main: Verificando archivo de entrada: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_restaurantes_con_ids.csv
2025-04-19 11:33:38 [INFO] __main__.main: Archivo de entrada encontrado.
2025-04-19 11:33:38 [INFO] __main__.cargar_datos_procesados: Cargando archivo procesado desde: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_restaurantes_con_ids.csv
2025-04-19 11:33:41 [INFO] __main__.cargar_datos_procesados: Archivo cargado: 277686 filas, 28 columnas.
2025-04-19 11:33:41 [INFO] __main__.main: --- Tarea 1: Convertir Nombres a Minúsculas y Guardar ---
2025-04-19 11:33:41 [INFO] __main__.nombres_a_minusculas_y_guardar: Convirtien